In [2]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('SQuAD/lib/python3.7/site-packages/en_core_web_sm/en_core_web_sm-2.2.5')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [1]:
import xgboost as xgb

In [3]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pickle
from textblob import TextBlob
from scipy import spatial
import torch
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import json

In [154]:
data = pd.read_csv("data/train_detect_sent_unsupervised_v3.csv").reset_index(drop=True)

In [155]:
data.shape

(86820, 14)

In [156]:
data.head(3)

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc,root_match_idx,root_match_idx_first
0,269.0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.04716371 0.08111103 ... 0.0...,"[0.6082267463207245, 0.7082076072692871, 0.669...","[8.0352955, 9.462915, 6.914433, 9.608288]",0,2,[],0
1,207.0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.05823731 0.07162362 ... 0.0...,"[0.5250032842159271, 0.51096311211586, 0.62473...","[6.9849253, 6.9543734, 6.4365134, 8.201017]",1,2,[],0
2,526.0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,3,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.04716371 0.11232486 ... 0.0...,"[0.4191964268684387, 0.4746186137199402, 0.584...","[5.770602, 6.6236463, 6.2470593, 7.1245747]",0,0,[],0


In [157]:
ast.literal_eval(data["sentences"][0])

['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress.',
 "Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.",
 "Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time.",
 'Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".']

In [158]:
data = data[data["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [159]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84346 entries, 0 to 84345
Data columns (total 14 columns):
answer_start            84346 non-null float64
context                 84346 non-null object
question                84346 non-null object
text                    84346 non-null object
sentences               84346 non-null object
target                  84346 non-null int64
sent_emb                84346 non-null object
quest_emb               84346 non-null object
cosine_sim              84346 non-null object
euclidean_dis           84346 non-null object
pred_idx_cos            84346 non-null int64
pred_idx_euc            84346 non-null int64
root_match_idx          84346 non-null object
root_match_idx_first    84346 non-null int64
dtypes: float64(1), int64(4), object(9)
memory usage: 9.0+ MB


In [160]:
def create_features(data):
    train = pd.DataFrame()
     
    for k in range(len(data["euclidean_dis"])):
        dis = ast.literal_eval(data["euclidean_dis"][k])
        for i in range(len(dis)):
            train.loc[k, "column_euc_"+"%s"%i] = dis[i]
    
    print("Finished")
    
    for k in range(len(data["cosine_sim"])):
        dis = ast.literal_eval(data["cosine_sim"][k].replace("nan","1"))
        for i in range(len(dis)):
            train.loc[k, "column_cos_"+"%s"%i] = dis[i]
            
    train["target"] = data["target"]
    return train

In [161]:
train = create_features(data)

Finished


In [162]:
del data

In [163]:
train.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,8.035296,9.462915,6.914433,9.608288,NaN,NaN,NaN,NaN,NaN,NaN,...,0.708208,0.669542,0.653088,NaN,NaN,NaN,NaN,NaN,NaN,1
1,6.984925,6.954373,6.436513,8.201017,NaN,NaN,NaN,NaN,NaN,NaN,...,0.510963,0.624731,0.549588,NaN,NaN,NaN,NaN,NaN,NaN,1
2,5.770602,6.623646,6.247059,7.124575,NaN,NaN,NaN,NaN,NaN,NaN,...,0.474619,0.584199,0.461928,NaN,NaN,NaN,NaN,NaN,NaN,3


In [11]:
# train.fillna(10000, inplace=True)

In [164]:
train.head(3).transpose()

,0,1,2
column_euc_0,8.035296,6.984925,5.770602
column_euc_1,9.462915,6.954373,6.623646
column_euc_2,6.914433,6.436513,6.247059
column_euc_3,9.608288,8.201017,7.124575
column_euc_4,NaN,NaN,NaN
column_euc_5,NaN,NaN,NaN
column_euc_6,NaN,NaN,NaN
column_euc_7,NaN,NaN,NaN
column_euc_8,NaN,NaN,NaN
column_euc_9,NaN,NaN,NaN


### Fitting Multinomial Logistic Regression

### Standardize

In [165]:
train.apply(max, axis = 0)

column_euc_0    23.558975
column_euc_1    40.290561
column_euc_2    33.286404
column_euc_3    34.140771
column_euc_4          NaN
column_euc_5          NaN
column_euc_6          NaN
column_euc_7          NaN
column_euc_8          NaN
column_euc_9          NaN
column_cos_0     1.411099
column_cos_1     1.489619
column_cos_2     1.397736
column_cos_3     1.490736
column_cos_4          NaN
column_cos_5          NaN
column_cos_6          NaN
column_cos_7          NaN
column_cos_8          NaN
column_cos_9          NaN
target           9.000000
dtype: float64

In [166]:
subset1 = train.iloc[:,:10].fillna(60)
subset2 = train.iloc[:,10:].fillna(1)

In [167]:
subset1.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9
0,8.035296,9.462915,6.914433,9.608288,60.0,60.0,60.0,60.0,60.0,60.0
1,6.984925,6.954373,6.436513,8.201017,60.0,60.0,60.0,60.0,60.0,60.0
2,5.770602,6.623646,6.247059,7.124575,60.0,60.0,60.0,60.0,60.0,60.0


In [168]:
subset2.head(3)

,column_cos_0,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,0.608227,0.708208,0.669542,0.653088,1.0,1.0,1.0,1.0,1.0,1.0,1
1,0.525003,0.510963,0.624731,0.549588,1.0,1.0,1.0,1.0,1.0,1.0,1
2,0.419196,0.474619,0.584199,0.461928,1.0,1.0,1.0,1.0,1.0,1.0,3


In [169]:
train2 = pd.concat([subset1, subset2],axis=1, join_axes=[subset1.index])

In [170]:
train2.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,8.035296,9.462915,6.914433,9.608288,60.0,60.0,60.0,60.0,60.0,60.0,...,0.708208,0.669542,0.653088,1.0,1.0,1.0,1.0,1.0,1.0,1
1,6.984925,6.954373,6.436513,8.201017,60.0,60.0,60.0,60.0,60.0,60.0,...,0.510963,0.624731,0.549588,1.0,1.0,1.0,1.0,1.0,1.0,1
2,5.770602,6.623646,6.247059,7.124575,60.0,60.0,60.0,60.0,60.0,60.0,...,0.474619,0.584199,0.461928,1.0,1.0,1.0,1.0,1.0,1.0,3


In [171]:
train2.apply(max, axis = 0)

column_euc_0    23.558975
column_euc_1    60.000000
column_euc_2    60.000000
column_euc_3    60.000000
column_euc_4    60.000000
column_euc_5    60.000000
column_euc_6    60.000000
column_euc_7    60.000000
column_euc_8    60.000000
column_euc_9    60.000000
column_cos_0     1.411099
column_cos_1     1.489619
column_cos_2     1.397736
column_cos_3     1.490736
column_cos_4     1.360776
column_cos_5     1.355459
column_cos_6     1.415495
column_cos_7     1.354246
column_cos_8     1.315012
column_cos_9     1.159472
target           9.000000
dtype: float64

In [172]:
scaler = MinMaxScaler()
X = scaler.fit_transform(train2.iloc[:,:-1])

In [173]:
X

array([[0.32986874, 0.15283914, 0.1061833 , ..., 0.71921987, 0.74207715,
        0.84381083],
       [0.28452602, 0.11078807, 0.09813643, ..., 0.71921987, 0.74207715,
        0.84381083],
       [0.23210571, 0.10524405, 0.09494654, ..., 0.71921987, 0.74207715,
        0.84381083],
       ...,
       [0.21586385, 0.0857103 , 0.10546501, ..., 0.71921987, 0.74207715,
        0.84381083],
       [0.22143519, 0.06897399, 0.10499792, ..., 0.71921987, 0.74207715,
        0.84381083],
       [0.29449538, 0.11801073, 0.16833386, ..., 0.71921987, 0.74207715,
        0.84381083]])

In [174]:
train_x, test_x, train_y, test_y = train_test_split(X,
train.iloc[:,-1], train_size=0.8, random_state = 5)

In [175]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_x, train_y)

print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))

Multinomial Logistic regression Train Accuracy :  0.6252593514731164
Multinomial Logistic regression Test Accuracy :  0.6289270895080024


### Logistic-Regression with Root Match feature

In [176]:
predicted = pd.read_csv("data/train_detect_sent_unsupervised_v3.csv").reset_index(drop=True)

In [177]:
predicted = predicted[predicted["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [178]:
predicted.shape

(84346, 14)

In [179]:
def get_columns_from_root(train):
    
    for i in range(train.shape[0]):
        if len(ast.literal_eval(train["root_match_idx"][i])) == 0: pass
        
        else:
            for item in ast.literal_eval(train["root_match_idx"][i]):
                train.loc[i, "column_root_"+"%s"%item] = 1
    return train

In [180]:
predicted = get_columns_from_root(predicted)

In [181]:
predicted.head(3).transpose()

,0,1,2
answer_start,269,207,526
context,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
question,When did Beyonce start becoming popular?,What areas did Beyonce compete in when she was...,When did Beyonce leave Destiny's Child and bec...
text,in the late 1990s,singing and dancing,2003
sentences,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...
target,1,1,3
sent_emb,"[array([ 0.00746889, -0.04449936, 0.11347561,...","[array([ 0.00746889, -0.04449936, 0.11347561,...","[array([ 0.00746889, -0.04449936, 0.11347561,..."
quest_emb,[[ 0.00746889 -0.04716371 0.08111103 ... 0.0...,[[ 0.00746889 -0.05823731 0.07162362 ... 0.0...,[[ 0.00746889 -0.04716371 0.11232486 ... 0.0...
cosine_sim,"[0.6082267463207245, 0.7082076072692871, 0.669...","[0.5250032842159271, 0.51096311211586, 0.62473...","[0.4191964268684387, 0.4746186137199402, 0.584..."
euclidean_dis,"[8.0352955, 9.462915, 6.914433, 9.608288]","[6.9849253, 6.9543734, 6.4365134, 8.201017]","[5.770602, 6.6236463, 6.2470593, 7.1245747]"


In [182]:
subset3 = predicted[["column_root_0","column_root_1","column_root_2","column_root_3","column_root_4","column_root_5",\
             "column_root_6","column_root_7","column_root_8","column_root_9"]]

In [183]:
subset3.fillna(0, inplace=True)

In [184]:
train3 = pd.concat([subset3, train2],axis=1, join_axes=[subset3.index])

In [185]:
train3.head(3).transpose()

,0,1,2
column_root_0,0.000000,0.000000,0.000000
column_root_1,0.000000,0.000000,0.000000
column_root_2,0.000000,0.000000,0.000000
column_root_3,0.000000,0.000000,0.000000
column_root_4,0.000000,0.000000,0.000000
column_root_5,0.000000,0.000000,0.000000
column_root_6,0.000000,0.000000,0.000000
column_root_7,0.000000,0.000000,0.000000
column_root_8,0.000000,0.000000,0.000000
column_root_9,0.000000,0.000000,0.000000


In [186]:
train3 = train3[["column_root_0","column_root_1","column_root_2","column_root_3","column_root_4","column_root_5",\
             "column_root_6","column_root_7","column_root_8","column_root_9", "column_cos_0","column_cos_1",\
           "column_cos_2","column_cos_3","column_cos_4","column_cos_5",\
             "column_cos_6","column_cos_7","column_cos_8","column_cos_9", "target"]]

In [187]:
train_x, test_x, train_y, test_y = train_test_split(train3.iloc[:,:-1],
train3.iloc[:,-1], train_size=0.8, random_state = 5)

In [188]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_x, train_y)

print("Multinomial Logistic regression Root Match Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Multinomial Logistic regression Root Match Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))

Multinomial Logistic regression Root Match Train Accuracy :  0.6383899460548936
Multinomial Logistic regression Root Match Test Accuracy :  0.6435684647302905


### Random Forest

In [189]:
rf = RandomForestClassifier(min_samples_leaf=8, n_estimators=60)
rf.fit(train_x, train_y)

print("Random Forest Train Accuracy : ", metrics.accuracy_score(train_y, rf.predict(train_x)))
print("Random Forest Test Accuracy : ", metrics.accuracy_score(test_y, rf.predict(test_x)))

Random Forest Train Accuracy :  0.6975961823463157
Random Forest Test Accuracy :  0.6428571428571429


### XgBoost

In [190]:
model = xgb.XGBClassifier()
param_dist = {"max_depth": [3,5,10],
              "min_child_weight" : [1,5,10],
              "learning_rate": [0.07, 0.1,0.2],
               }

# run randomized search
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=5, n_jobs=-1)
grid_search.fit(train_x, train_y)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   52.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 17.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'learning_rate': [0.07, 0.1, 0.2],
                         'max_depth': [3, 5, 10],
                         'min_child_weight': [1, 5, 10]},
 

In [191]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=5, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [192]:
xg = xgb.XGBClassifier(max_depth=5)
xg.fit(train_x, train_y)

print("XGBoost Train Accuracy : ", metrics.accuracy_score(train_y, xg.predict(train_x)))
print("XGBoost Test Accuracy : ", metrics.accuracy_score(test_y, xg.predict(test_x)))

XGBoost Train Accuracy :  0.6761218803722805
XGBoost Test Accuracy :  0.6553645524599881


In [69]:
xgb.XGBClassifier??